In [14]:
import glob
import os
import os.path as osp
import numpy as np
from torch.utils.data import Dataset, DataLoader
from PIL import Image

from utils import convert_nii_3d, get_trainable_slices
from wrapper import OASIS
from split import split_data

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [29]:
scans_home = 'data/scans'
labels_file = 'data/OASIS3_MRID2Label_052918.csv'

In [30]:
train_filenames, val_filenames, test_filenames = split_data(scans_home, labels_file)

OSError: data/OASIS3_MRID2Label_052918.csv not found.

In [5]:
print(len(train_filenames))
print(len(val_filenames))
print(len(test_filenames))

1072
134
134


In [7]:
train_dataset = OASIS('scans', train_filenames, preload = True)
val_dataset = OASIS('scans', val_filenames, preload = True)
test_dataset = OASIS('scans', test_filenames, preload = True)

finished preloading
finished preloading
finished preloading


In [10]:
image_array, label = train_dataset[4]
print(image_array.shape)
print(len(train_dataset))
print(len(val_dataset))
print(len(test_dataset))
save_path = 'test_preview_scan/'

(256, 256)
39720
4934
4865


In [25]:
# get np H, W, D image
# img = convert_nii_3d('data/scans/OAS30001_MR_d0129/anat1/sub-OAS30001_sess-d0129_acq-TSE_T2w.nii.gz') #preview_scan
# img = convert_nii_3d('data/scans/OAS30001_MR_d2430/anat2/sub-OAS30001_sess-d2430_acq-TSE_T2w.nii.gz') #preview_scan_anat2_TSE
img = convert_nii_3d('data/scans/OAS30001_MR_d0129/anat4/sub-OAS30001_sess-d0129_T2w.nii.gz') #preview_scan_non_TSE
print(img.shape)
# loop over indices and print out mean of slice
for i in range(img.shape[2]):
    print(i, img[:,:,i].shape, img[:,:,i].mean())

(176, 256, 256)
0 (176, 256) 13.207719282670455
1 (176, 256) 13.121515447443182
2 (176, 256) 12.992143110795455
3 (176, 256) 12.850719105113637
4 (176, 256) 12.939630681818182
5 (176, 256) 12.766091086647727
6 (176, 256) 12.757990056818182
7 (176, 256) 12.720237038352273
8 (176, 256) 12.724631569602273
9 (176, 256) 12.870183771306818
10 (176, 256) 12.981378728693182
11 (176, 256) 13.120738636363637
12 (176, 256) 13.229048295454545
13 (176, 256) 13.330588600852273
14 (176, 256) 13.411243785511363
15 (176, 256) 13.472034801136363
16 (176, 256) 13.440585049715908
17 (176, 256) 13.483465021306818
18 (176, 256) 13.597256747159092
19 (176, 256) 13.784579190340908
20 (176, 256) 13.730668501420455
21 (176, 256) 13.693692294034092
22 (176, 256) 13.711181640625
23 (176, 256) 13.746271306818182
24 (176, 256) 13.904052734375
25 (176, 256) 13.990434126420455
26 (176, 256) 14.110040838068182
27 (176, 256) 14.169011896306818
28 (176, 256) 14.306285511363637
29 (176, 256) 14.340753728693182
30 (176, 2

In [10]:
# print out mean of all slices using np.mean (axis)
print(img.shape)
print(img.reshape(-1, img.shape[2]).mean(axis=0))

(256, 256, 36)
[18.83564758 19.6512146  18.93692017 17.61628723 17.56167603 17.4723053
 16.92080688 16.72193909 17.16856384 17.39790344 17.55918884 17.62760925
 17.91520691 18.4209137  19.61613464 19.13951111 18.76333618 17.8394165
 17.49301147 17.33889771 17.54119873 17.26580811 17.24360657 16.1815033
 16.3401947  16.03501892 15.74476624 14.80212402 14.42456055 13.15753174
 12.49742126 11.61732483 10.76405334  7.98591614  5.83325195  3.88075256]


In [19]:
#print out max mean of all slices using mean max
print(img.reshape(-1, img.shape[2]).mean(axis=0).max())

32.42316228693182


In [41]:
# get all images that are above a threshold
THRESHOLD_VAL = 0.75
threshold = img.reshape(-1, img.shape[2]).mean(axis=0).max() * THRESHOLD_VAL
print(np.arange(img.shape[2])[img.reshape(-1, img.shape[2]).mean(axis=0) > threshold])

[ 93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108 109 110
 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128
 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146
 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162]


In [26]:
print(np.arange(img.shape[2])[get_trainable_slices(img)])

[ 89  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106
 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124
 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142
 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160
 161 162 163 164 165 166]
